In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [2]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [3]:
def parse_data(input: str) -> List[str]:
    return re.findall(r"nw|se|ne|sw|e|w", input)

In [4]:
mini_test = "sesenwnenenewseeswwswswwnenewsewsw"
assert len("".join(parse_data(mini_test))) == len(mini_test)

In [5]:
test_string = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
"""
test_ins = read_data(test_string, parser=parse_data, sep=None, testing=True)

Part I  

Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, how many tiles are left with the black side up?

In [6]:
dir = dict(
    e=(1,0),
    w=(-1,0),
    nw=(0,-1),
    se=(0,1),
    ne=(1,-1),
    sw=(-1,1)
)

def get_final_coord(ins: List[str]) -> Tuple[int]:
    x, y = (0, 0)
    for instruct in ins:
        dx, dy = dir[instruct]
        x += dx
        y += dy
    return (x, y)

def get_blacks(ins: List[List[str]]) -> Set:
    blacks = set()
    for tile in ins:
        tile_coord = get_final_coord(tile)
        blacks.remove(tile_coord) if tile_coord in blacks else blacks.add(tile_coord)
    return blacks

def run_part1(ins: List[List[str]]) -> int:
    blacks = get_blacks(ins)
    return len(blacks)

In [7]:
run_part1(test_ins)

10

In [8]:
real_ins = read_data("input.txt", parser=parse_data, sep=None)
run_part1(real_ins)

289

Part II

How many tiles will be black after 100 days?

In [9]:
criteria = dict(
    black=(1, 2),
    white=(2,)
)

def get_neighbors(tile: Tuple[int]) -> List[Tuple[int]]:
    x, y = tile
    return [(x+dx, y+dy) for dx, dy in dir.values()]

def run_part2(ins: List[List[str]], days=100) -> int:
    # get the starting blacks
    blacks = get_blacks(ins)
    # iterate over days
    for _ in range(days):
        # count the number of black neighbors for each tile
        counts = Counter(chain(*[get_neighbors(tile) for tile in blacks]))
        # evaluate the new blacks for next iteration
        blacks = {tile for tile, count in counts.items() if tile in blacks and count in criteria["black"]}.union({tile for tile, count in counts.items() if tile not in blacks and count in criteria["white"]})

    return len(blacks)


In [10]:
run_part2(test_ins)

2208

In [11]:
run_part2(real_ins)

3551